In [ ]:
%matplotlib inline

import sys
sys.path.append('../')

import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib

from src.utils import plot_list, read_images
from src.metrics import compute_ious, compute_eval_metric

VALIDATION_RESULTS_FILEPATH = 'YOUR/validation_results.pkl'

In [ ]:
meta_valid, y_true, y_pred = joblib.load(VALIDATION_RESULTS_FILEPATH)

In [ ]:
meta_valid.head()

In [ ]:
ious = [compute_ious(gt, pred)[0][0] for gt, pred in zip(y_true, y_pred)]
iouts = [compute_eval_metric(gt, pred) for gt, pred in zip(y_true, y_pred)]

raw_imgs = read_images(meta_valid['file_path_image'])
depths = meta_valid['z'].tolist()
results = list(zip(ious, iouts, depths, raw_imgs, y_true, y_pred))

# Score distributions

In [ ]:
sns.distplot(ious)

In [ ]:
sns.distplot(iouts)

It seems that the model is either really good or really bad.
For example:

In [ ]:
print(np.mean([score for score in iouts if score>0.1]))

# score by depth

In [ ]:
sns.regplot(depths, iouts, fit_reg=False)

It seems that the model is better for lower depths.
Not sure yet what to do with it.

# Predicted mask exploration

In [ ]:
def filter_results(results, iout_range):
    iout_min, iout_max = iout_range
    results_filtered = []
    for tup in results:
        if iout_min<=tup[1]<=iout_max:
            results_filtered.append(tup)
    return results_filtered

results_filtered = filter_results(results, iout_range=(0.0,0.5))
len(results_filtered)

In [ ]:
np.mean(list(zip(*results))[2]), np.mean(list(zip(*results_filtered))[2])

In [ ]:
IMG_NR = 77

for iou, iout, z, img, gt, pred in results_filtered[:IMG_NR]:
    print('IOU {}, IOUT {}, depth {}'.format(iou, iout, z))
    plot_list(images=[img],labels=[gt, pred])